In [1]:
#!pip install --upgrade pip
#!pip install numpy
#!pip install pandas
#!pip install xlrd
debug = 2
import numpy as np
import pandas
import time
import geopandas
import os
pandas.set_option('display.max_colwidth', -1)


# Category should be one of: 
##  demographics, ancestry, birthplace, residence, transportation, household, education...
##  language, poverty, disability, income, veteran, employment, housing, health, qa

category = 'qa'
for_cra_analysis_mac_varlist = ['B01001_001', 'B08013_001', 'B02001_003', 'B02001_004', 'B02001_008', 'B03001_003', 'B15001_001', 'B17001_001', 'B19013_001', 'B19062_001']

needed_variables = ['FILEID', 'FILETYPE', 'STUSAB', 'CHARITER', 'SEQUENCE', 'LOGRECNO', 'B00001_001', 'B00002_001']
needed_variables = needed_variables + for_cra_analysis_mac_varlist


basedir = '/home/idies/workspace/Temporary/raddick/census_scratch/acs5/'
rawdatadir = basedir + 'rawdata/'
metadir = basedir + 'metadata/xls_temp/'

estimates_dir = basedir + 'estimates/'
margin_of_error_dir = basedir + 'margin_of_error/'
vardir = basedir + 'variables/'

geodir = basedir + 'geography/'
#shapefiledir = basedir + 'geography/TRACT_SHAPEFILES/'

for thisdir in [estimates_dir, margin_of_error_dir, vardir, geodir]:#[datadir, errordir, vardir, geodir]:
    if not(os.path.exists(thisdir)):
        os.makedirs(thisdir)
        
state_codes_df = pandas.read_csv('extras/statecodes.csv')
state_codes_df = state_codes_df.set_index('STUSAB')
#print('found state codes')
if (debug >=  1):
    print('Category: {0}'.format(category))
print('Done!')



Category: qa
Done!


In [2]:
estimates_df = pandas.DataFrame()
metadata_df = pandas.DataFrame()
want_sequences = []

for i in range(1,3):
    want_sequences.append(i)  

if (category == 'for_cra_analysis_mac'):
    for i in [4, 5, 23, 43, 48, 59, 63]:
        want_sequences.append(i)
elif (category == 'demographics'):
    for i in range(3,6):
        want_sequences.append(i)
elif (category == 'ancestry'):
    for i in range(6,8):
        want_sequences.append(i)
elif (category == 'birthplace'):
    for i in range(8,16):
        want_sequences.append(i)
elif (category == 'residence'):
    for i in range(16,23):
        want_sequences.append(i)
elif (category == 'transportation'):
    for i in range(23,34):
        want_sequences.append(i)
elif (category == 'household'):
    for i in range(34,41):
        want_sequences.append(i)
elif (category == 'education'):
    for i in range(41,45):
        want_sequences.append(i)
elif (category == 'language'):
    for i in range(45,48):
        want_sequences.append(i)
elif (category == 'poverty'):
    for i in range(48,57):
        want_sequences.append(i)
elif (category == 'disability'):
    for i in range(57,59):
        want_sequences.append(i)
elif (category == 'income'):
    for i in range(59,73):
        want_sequences.append(i)
elif (category == 'veteran'):
    for i in range(73,75):
        want_sequences.append(i)
elif (category == 'employment'):
    for i in range(75,103):
        want_sequences.append(i)
elif (category == 'housing'):
    for i in range(103,114):
        want_sequences.append(i)     
elif (category == 'health'):
    for i in range(114,118):
        want_sequences.append(i)     
elif (category == 'qa'):
    for i in range(118,123):
        want_sequences.append(i)     

states = state_codes_df.index.values.tolist()
states = [x.lower() for x in states if x not in ('AS', 'GU', 'MP', 'PR', 'UM', 'VI')]

for i in want_sequences:
    if (debug >= 1):
        print('Importing estimates for sequence {0:,.0f}...'.format(i))

    this_seq_metadata_filename = metadir + 'seq{0:.0f}.xlsx'.format(i)
    this_seq_metadata_in_cols_df = pandas.read_excel(this_seq_metadata_filename, header=None, encoding='utf-8')
    this_seq_metadata_df = this_seq_metadata_in_cols_df.T
    this_seq_metadata_in_cols_df.columns = this_seq_metadata_in_cols_df.loc[0]

    this_seq_metadata_df = this_seq_metadata_in_cols_df.T
    this_seq_metadata_df.columns = ['variable', 'description']
    
    if (i == 1):
        this_seq_metadata_df = this_seq_metadata_df.assign(sequence_number = np.nan)
        this_seq_metadata_df['sequence_number'] = [0,0,0,0,0,0,1,1]
    else:
        this_seq_metadata_df = this_seq_metadata_df.assign(sequence_number = i)
    
    this_seq_estimates_df = pandas.DataFrame()
    for onestate in states:
        statefilename = rawdatadir + 'e20175{1:s}{2:04d}000.txt'.format(i,onestate,i)
        onestate_df = pandas.read_csv(statefilename, header=None, sep=',', encoding='utf-8', low_memory=False)
        this_seq_estimates_df = this_seq_estimates_df.append(onestate_df)
    
    this_seq_estimates_df.columns = this_seq_metadata_in_cols_df.columns   
    
    if (i >= 2):
        this_seq_metadata_df = this_seq_metadata_df.drop(['FILEID','FILETYPE','STUSAB', 'SEQUENCE', 'CHARITER','LOGRECNO'], axis=0)
        this_seq_estimates_df = this_seq_estimates_df.drop(['FILEID','FILETYPE','STUSAB', 'CHARITER', 'LOGRECNO'], axis=1)

    if (debug >= 2):
        print('Merging datasets...')
    metadata_df = pandas.concat((metadata_df, this_seq_metadata_df), axis=0)
    estimates_df = pandas.concat((estimates_df, this_seq_estimates_df), axis=1)

estimates_df['FILETYPE'] = estimates_df['FILETYPE'].astype('str')
estimates_df['STUSAB'] = estimates_df['STUSAB'].astype('str')


#print('Keeping only needed variables...')
#data_df = data_df[needed_variables]
#metadata_df = metadata_df.loc[needed_variables]

print('backing up...')
estimates_df_bk = estimates_df
print('Done!')
#metadata_df.T

Importing estimates for sequence 1...
Merging datasets...
Importing estimates for sequence 2...
Merging datasets...
Importing estimates for sequence 118...
Merging datasets...
Importing estimates for sequence 119...
Merging datasets...
Importing estimates for sequence 120...
Merging datasets...
Importing estimates for sequence 121...
Merging datasets...
Importing estimates for sequence 122...
Merging datasets...
backing up...
Done!


In [3]:
margin_of_error_df = pandas.DataFrame()
metadata_df = pandas.DataFrame()
want_sequences = []

for i in range(1,3):
    want_sequences.append(i)  

if (category == 'for_cra_analysis_mac'):
    for i in [4, 5, 23, 43, 48, 59, 63]:
        want_sequences.append(i)
elif (category == 'demographics'):
    for i in range(3,6):
        want_sequences.append(i)
elif (category == 'ancestry'):
    for i in range(6,8):
        want_sequences.append(i)
elif (category == 'birthplace'):
    for i in range(8,16):
        want_sequences.append(i)
elif (category == 'residence'):
    for i in range(16,23):
        want_sequences.append(i)
elif (category == 'transportation'):
    for i in range(23,34):
        want_sequences.append(i)
elif (category == 'household'):
    for i in range(34,41):
        want_sequences.append(i)
elif (category == 'education'):
    for i in range(41,45):
        want_sequences.append(i)
elif (category == 'language'):
    for i in range(45,48):
        want_sequences.append(i)
elif (category == 'poverty'):
    for i in range(48,57):
        want_sequences.append(i)
elif (category == 'disability'):
    for i in range(57,59):
        want_sequences.append(i)
elif (category == 'income'):
    for i in range(59,73):
        want_sequences.append(i)
elif (category == 'veteran'):
    for i in range(73,75):
        want_sequences.append(i)
elif (category == 'employment'):
    for i in range(75,103):
        want_sequences.append(i)
elif (category == 'housing'):
    for i in range(103,114):
        want_sequences.append(i)     
elif (category == 'health'):
    for i in range(114,118):
        want_sequences.append(i)     
elif (category == 'qa'):
    for i in range(118,123):
        want_sequences.append(i)     

states = state_codes_df.index.values.tolist()
states = [x.lower() for x in states if x not in ('AS', 'GU', 'MP', 'PR', 'UM', 'VI')]

for i in want_sequences:
    if (debug >= 1):
        print('Importing margin of error values for sequence {0:,.0f}...'.format(i))

    this_seq_metadata_filename = metadir + 'seq{0:.0f}.xlsx'.format(i)
    this_seq_metadata_in_cols_df = pandas.read_excel(this_seq_metadata_filename, header=None, encoding='utf-8')
    this_seq_metadata_df = this_seq_metadata_in_cols_df.T
    this_seq_metadata_in_cols_df.columns = this_seq_metadata_in_cols_df.loc[0]

    this_seq_metadata_df = this_seq_metadata_in_cols_df.T
    this_seq_metadata_df.columns = ['variable', 'description']
    
    if (i == 1):
        this_seq_metadata_df = this_seq_metadata_df.assign(sequence_number = np.nan)
        this_seq_metadata_df['sequence_number'] = [0,0,0,0,0,0,1,1]
    else:
        this_seq_metadata_df = this_seq_metadata_df.assign(sequence_number = i)
    
    this_seq_margin_of_error_df = pandas.DataFrame()
    for onestate in states:
        statefilename = rawdatadir + 'm20175{1:s}{2:04d}000.txt'.format(i,onestate,i)
        onestate_df = pandas.read_csv(statefilename, header=None, sep=',', encoding='utf-8', low_memory=False)
        this_seq_margin_of_error_df = this_seq_margin_of_error_df.append(onestate_df)
        
    
    this_seq_margin_of_error_df.columns = this_seq_metadata_in_cols_df.columns   
#    print('this seq error file has {0:,.0f} rows'.format(len(this_seq_error_df)))    
#    print(this_seq_error_df.sample(1).T)
    
    if (i >= 2):
        this_seq_metadata_df = this_seq_metadata_df.drop(['FILEID','FILETYPE','STUSAB', 'SEQUENCE', 'CHARITER','LOGRECNO'], axis=0)
        this_seq_margin_of_error_df = this_seq_margin_of_error_df.drop(['FILEID','FILETYPE','STUSAB', 'CHARITER', 'LOGRECNO'], axis=1)

    if (debug >= 2):
        print('Merging datasets...')
    metadata_df = pandas.concat((metadata_df, this_seq_metadata_df), axis=0)
    margin_of_error_df = pandas.concat((margin_of_error_df, this_seq_margin_of_error_df), axis=1)

#error_df.sample(1).T
margin_of_error_df['FILETYPE'] = margin_of_error_df['FILETYPE'].astype('str')
margin_of_error_df['STUSAB'] = margin_of_error_df['STUSAB'].astype('str')

#print('Keeping only needed variables...')
#error_df = error_df[needed_variables]
#metadata_df = metadata_df.loc[needed_variables]

print('backing up...')
margin_of_error_df_bk = margin_of_error_df
print('Done!')
#margin_of_error_df.sample(1)
#metadata_df.T

Importing margin of error values for sequence 1...
Merging datasets...
Importing margin of error values for sequence 2...
Merging datasets...
Importing margin of error values for sequence 118...
Merging datasets...
Importing margin of error values for sequence 119...
Merging datasets...
Importing margin of error values for sequence 120...
Merging datasets...
Importing margin of error values for sequence 121...
Merging datasets...
Importing margin of error values for sequence 122...
Merging datasets...
backing up...
Done!


In [4]:
s = time.time()
geo_df = pandas.DataFrame()

states = state_codes_df.index.values.tolist()
states = [x.lower() for x in states if x not in ('AS', 'GU', 'MP', 'PR', 'UM', 'VI')]

if (debug >= 1):
        print('Reading geography files...')
for onestate in states:
    filename = geodir+'{0:}.xlsx'.format(onestate)
    if (debug == 2):
        print('Reading geography for {0:}...'.format(onestate))
    this_geo_df = pandas.read_excel(filename)
    geo_df = pandas.concat((geo_df, this_geo_df))

geo_df = geo_df.assign(STUSAB = geo_df['State'].apply(lambda x: x.lower()))

geo_df = geo_df.rename(columns={'Geography ID': 'GEOID'})

if (debug >= 1):
    print('Retaining only tractlevel geographies...')
##    print('Retaining only tract- and block-group-level geographies...')
geo_df = geo_df[(geo_df['GEOID'].apply(lambda x: x[0:3] == '140'))]
##geo_df = geo_df[(geo_df['GEOID'].apply(lambda x: x[0:3] == '140')) | (geo_df['GEOID'].apply(lambda x: x[0:3] == '150'))]

print('backing up...')
geo_df_bk = geo_df

##geo_df = geo_df.set_index('GEOID')  #We'll set GEOID as index colum AFTER the merge
e = time.time()
print('Done in {0:.1f} seconds!'.format(e-s))

#geo_df.sample(1)
#print('skipping...')

Reading geography files...
Reading geography for al...
Reading geography for ak...
Reading geography for az...
Reading geography for ar...
Reading geography for ca...
Reading geography for co...
Reading geography for ct...
Reading geography for de...
Reading geography for dc...
Reading geography for fl...
Reading geography for ga...
Reading geography for hi...
Reading geography for id...
Reading geography for il...
Reading geography for in...
Reading geography for ia...
Reading geography for ks...
Reading geography for ky...
Reading geography for la...
Reading geography for me...
Reading geography for md...
Reading geography for ma...
Reading geography for mi...
Reading geography for mn...
Reading geography for ms...
Reading geography for mo...
Reading geography for mt...
Reading geography for ne...
Reading geography for nv...
Reading geography for nh...
Reading geography for nj...
Reading geography for nm...
Reading geography for ny...
Reading geography for nc...
Reading geography for

In [5]:
print('getting from backup...')
estimates_df = estimates_df_bk
margin_of_error_df = margin_of_error_df_bk

#.sort_values('Logical Record Number')
#data_df[['STUSAB','LOGRECNO']].dtypes #object, int64
#geo_df[['STUSAB', 'Logical Record Number']].dtypes
estimates_df = estimates_df.merge(geo_df, left_on=['STUSAB', 'LOGRECNO'], right_on=['STUSAB', 'Logical Record Number'])
estimates_df = estimates_df.set_index('GEOID')  # set the GEOID after we add the shapefiles

margin_of_error_df = margin_of_error_df.merge(geo_df, left_on=['STUSAB', 'LOGRECNO'], right_on=['STUSAB', 'Logical Record Number'])
margin_of_error_df = margin_of_error_df.set_index('GEOID')  # set the GEOID after we add the shapefiles

print('backing up...')
estimates_df_bk = estimates_df
margin_of_error_df_bk = margin_of_error_df

print('Total estimate of B01001_001: {0:,.0f}'.format(estimates_df['B01001_001'].sum()))
print('Margin of error in B01001_001: {0:,.0f}'.format(margin_of_error_df['B01001_001'].sum()))
print('Done!')


getting from backup...
backing up...
Total estimate of B01001_001: 321,004,407
Margin of error in B01001_001: 26,621,941
Done!


In [6]:
#data_df = data_df[varlist]
s = time.time()
print('Writing data...')

estimates_df.to_csv(estimates_dir + 'estimates_acs2017_tract_{0:}.csv'.format(category), encoding='utf-8')
print('Estimates written to estimates_acs2017_tract_{0:}.csv...'.format(category))
margin_of_error_df.to_csv(margin_of_error_dir + 'margin_of_error_acs2017_tract_{0:}.csv'.format(category), encoding='utf-8')
print('Margins of error written to margin_of_error_acs2017_tract_{0:}.csv...'.format(category))

metadata_df.to_csv(vardir + 'variables_acs2017_tract_{0:}.csv'.format(category), encoding='utf-8')
#geo_df.to_csv(geodir + 'geo_acs2017_tract.csv', encoding='utf-8')
e = time.time()
print('Done in {0:,.0f} minutes {1:.0f} seconds!'.format(np.round((e-s)/60,0), ((e-s)%60)))

Writing data...
Estimates written to estimates_acs2017_tract_qa.csv...
Margins of error written to margin_of_error_acs2017_tract_qa.csv...
Done in 4 minutes 11 seconds!
